In [ ]:
import pandas as pd
import glob
import os

# Caminhos
path_fbref = r"d:/Diogo/Ambiente de Trabalho/PROJETO/Datasets/FBRef/dataset_completo_final.csv"
path_fd = r"d:/Diogo/Ambiente de Trabalho/PROJETO/Datasets/football_data_co_uk/"

# Carregar FBRef
df_fbref = pd.read_csv(path_fbref)

# Carregar Jogos
files = glob.glob(os.path.join(path_fd, "P1_*.csv"))
dfs = [pd.read_csv(f) for f in files]
df_matches = pd.concat(dfs, ignore_index=True)

# Tratar datas e remover lixo
df_matches['Date'] = pd.to_datetime(df_matches['Date'], dayfirst=True, errors='coerce')
df_matches = df_matches.dropna(subset=['Date']).copy().sort_values('Date')
df_matches = df_matches.loc[:, ~df_matches.columns.str.contains('^Unnamed')]

print("Dados carregados e limpos.")

In [ ]:
# Mapeamento de nomes
mapping = {
    "AVS": "AVS Futebol", "Academica": "Académica", "Beira Mar": "Beira-Mar",
    "Famalicao": "Famalicão", "Maritimo": "Marítimo", "Pacos Ferreira": "Paços de Ferreira",
    "Sp Braga": "Braga", "Sp Lisbon": "Sporting CP", "Setubal": "Vitória Setúbal",
    "Uniao Madeira": "União", "Guimaraes": "Vitória", "Gil Vicente": "Gil Vicente",
    "Nacional": "Nacional", "Estrela": "Estrela", "Leiria": "União de Leiria",
    "Leixoes": "Leixões", "Feirense": "Feirense", "Belenenses": "Belenenses"
}

df_matches['HomeTeam'] = df_matches['HomeTeam'].str.strip().replace(mapping)
df_matches['AwayTeam'] = df_matches['AwayTeam'].str.strip().replace(mapping)

df_fbref['Equipa'] = df_fbref['Equipa'].replace({
    "Gil Vicente FC": "Gil Vicente", "B-SAD": "Belenenses", "Sporting": "Sporting CP"
})

print("Nomes corrigidos.")

In [ ]:
# Calcular épocas
df_matches['Season'] = df_matches['Date'].apply(lambda x: f"{x.year}-{x.year+1}" if x.month >= 8 else f"{x.year-1}-{x.year}")
df_matches['Previous_Season'] = df_matches['Season'].apply(lambda x: f"{int(x.split('-')[0])-1}-{int(x.split('-')[0])}")

# Preparar dados históricos (FBRef)
cols_renomeadas = {c: f"hist_{c}" for c in df_fbref.columns if c not in ['Equipa', 'Epoca']}
df_hist = df_fbref.rename(columns=cols_renomeadas)

# --- IMPUTAÇÃO: Calcular média das 3 piores (Relegation Zone) ---
relegation_stats = []
for season in df_hist['Epoca'].unique():
    season_data = df_hist[df_hist['Epoca'] == season].sort_values('hist_Pontos')
    if not season_data.empty:
        bottom_3 = season_data.head(3).mean(numeric_only=True)
        bottom_3['Epoca'] = season
        relegation_stats.append(bottom_3)

df_relegation = pd.DataFrame(relegation_stats)

# Merge Inicial
df_final = pd.merge(df_matches, df_hist, left_on=['HomeTeam', 'Previous_Season'], right_on=['Equipa', 'Epoca'], how='left')
df_final = df_final.rename(columns={c: f"Home_{c}" for c in df_hist.columns if 'hist_' in c})

df_final = pd.merge(df_final, df_hist, left_on=['AwayTeam', 'Previous_Season'], right_on=['Equipa', 'Epoca'], how='left', suffixes=('', '_Away'))
df_final = df_final.rename(columns={c: f"Away_{c}" for c in df_hist.columns if 'hist_' in c})

df_final = df_final.drop(columns=['Equipa', 'Epoca', 'Equipa_Away', 'Epoca_Away'], errors='ignore')

# Aplicar Imputação
for prefix in ['Home_', 'Away_']:
    df_defaults = pd.merge(df_final[['Previous_Season']], df_relegation, left_on='Previous_Season', right_on='Epoca', how='left')
    rename_dict = {c: f"{prefix}{c}" for c in df_relegation.columns if c != 'Epoca'}
    df_defaults = df_defaults.rename(columns=rename_dict)
    
    for col in rename_dict.values():
        if col in df_final.columns:
            df_final[col] = df_final[col].fillna(df_defaults[col])

print(f"Merge e Imputação feitos. Shape: {df_final.shape}")

In [23]:
# --- RENOMEAR COLUNAS PARA PORTUGUÊS (COMPLETO) ---
col_map_pt = {
    # Info Jogo
    'Div': 'Divisao',
    'Date': 'Data', 'Time': 'Hora', 'HomeTeam': 'Equipa_Casa', 'AwayTeam': 'Equipa_Visitante',
    'FTHG': 'Golos_Casa_Final', 'FTAG': 'Golos_Visitante_Final', 'FTR': 'Resultado_Final',
    'HTHG': 'Golos_Casa_Intervalo', 'HTAG': 'Golos_Visitante_Intervalo', 'HTR': 'Resultado_Intervalo',
    'HS': 'Remates_Casa', 'AS': 'Remates_Visitante', 'HST': 'Remates_Alvo_Casa', 'AST': 'Remates_Alvo_Visitante',
    'HF': 'Faltas_Casa', 'AF': 'Faltas_Visitante', 'HC': 'Cantos_Casa', 'AC': 'Cantos_Visitante',
    'HY': 'Amarelos_Casa', 'AY': 'Amarelos_Visitante', 'HR': 'Vermelhos_Casa', 'AR': 'Vermelhos_Visitante',
    
    # Odds 1X2 (Principais)
    'B365H': 'Odd_Casa_Bet365', 'B365D': 'Odd_Empate_Bet365', 'B365A': 'Odd_Visitante_Bet365',
    'AvgH': 'Odd_Casa_Media', 'AvgD': 'Odd_Empate_Media', 'AvgA': 'Odd_Visitante_Media',
    'MaxH': 'Odd_Casa_Maxima', 'MaxD': 'Odd_Empate_Maxima', 'MaxA': 'Odd_Visitante_Maxima',
    'PSH': 'Odd_Casa_Pinnacle', 'PSD': 'Odd_Empate_Pinnacle', 'PSA': 'Odd_Visitante_Pinnacle',
    
    # Odds 1X2 (Outras Casas)
    'BWH': 'Odd_Casa_BetWin', 'BWD': 'Odd_Empate_BetWin', 'BWA': 'Odd_Visitante_BetWin',
    'GBH': 'Odd_Casa_Gamebookers', 'GBD': 'Odd_Empate_Gamebookers', 'GBA': 'Odd_Visitante_Gamebookers',
    'IWH': 'Odd_Casa_Interwetten', 'IWD': 'Odd_Empate_Interwetten', 'IWA': 'Odd_Visitante_Interwetten',
    'LBH': 'Odd_Casa_Ladbrokes', 'LBD': 'Odd_Empate_Ladbrokes', 'LBA': 'Odd_Visitante_Ladbrokes',
    'SBH': 'Odd_Casa_Sportingbet', 'SBD': 'Odd_Empate_Sportingbet', 'SBA': 'Odd_Visitante_Sportingbet',
    'WHH': 'Odd_Casa_WilliamHill', 'WHD': 'Odd_Empate_WilliamHill', 'WHA': 'Odd_Visitante_WilliamHill',
    'SJH': 'Odd_Casa_StanJames', 'SJD': 'Odd_Empate_StanJames', 'SJA': 'Odd_Visitante_StanJames',
    'VCH': 'Odd_Casa_VCBet', 'VCD': 'Odd_Empate_VCBet', 'VCA': 'Odd_Visitante_VCBet',
    'BSH': 'Odd_Casa_BlueSquare', 'BSD': 'Odd_Empate_BlueSquare', 'BSA': 'Odd_Visitante_BlueSquare',
    'BFH': 'Odd_Casa_Betfair', 'BFD': 'Odd_Empate_Betfair', 'BFA': 'Odd_Visitante_Betfair',
    '1XBH': 'Odd_Casa_1xBet', '1XBD': 'Odd_Empate_1xBet', '1XBA': 'Odd_Visitante_1xBet',
    'BMGMH': 'Odd_Casa_BetMGM', 'BMGMD': 'Odd_Empate_BetMGM', 'BMGMA': 'Odd_Visitante_BetMGM',
    'BVH': 'Odd_Casa_BetVictor', 'BVD': 'Odd_Empate_BetVictor', 'BVA': 'Odd_Visitante_BetVictor',
    'CLH': 'Odd_Casa_Coral', 'CLD': 'Odd_Empate_Coral', 'CLA': 'Odd_Visitante_Coral',
    
    # Betbrain (Agregadores Antigos)
    'Bb1X2': 'Num_Casas_Betbrain',
    'BbMxH': 'Odd_Casa_Maxima_Betbrain', 'BbAvH': 'Odd_Casa_Media_Betbrain',
    'BbMxD': 'Odd_Empate_Maxima_Betbrain', 'BbAvD': 'Odd_Empate_Media_Betbrain',
    'BbMxA': 'Odd_Visitante_Maxima_Betbrain', 'BbAvA': 'Odd_Visitante_Media_Betbrain',
    'BbOU': 'Num_Casas_OverUnder_Betbrain',
    'BbMx>2.5': 'Odd_Mais2.5_Maxima_Betbrain', 'BbAv>2.5': 'Odd_Mais2.5_Media_Betbrain',
    'BbMx<2.5': 'Odd_Menos2.5_Maxima_Betbrain', 'BbAv<2.5': 'Odd_Menos2.5_Media_Betbrain',
    'BbAH': 'Num_Casas_Handicap_Betbrain', 'BbAHh': 'Handicap_Asiatico_Linha_Betbrain',
    'BbMxAHH': 'Odd_Handicap_Casa_Maxima_Betbrain', 'BbAvAHH': 'Odd_Handicap_Casa_Media_Betbrain',
    'BbMxAHA': 'Odd_Handicap_Visitante_Maxima_Betbrain', 'BbAvAHA': 'Odd_Handicap_Visitante_Media_Betbrain',

    # Odds Handicap Asiático
    'AHh': 'Handicap_Asiatico_Linha',
    'B365AHH': 'Odd_Handicap_Casa_Bet365', 'B365AHA': 'Odd_Handicap_Visitante_Bet365',
    'PAHH': 'Odd_Handicap_Casa_Pinnacle', 'PAHA': 'Odd_Handicap_Visitante_Pinnacle',
    'MaxAHH': 'Odd_Handicap_Casa_Maxima', 'MaxAHA': 'Odd_Handicap_Visitante_Maxima',
    'AvgAHH': 'Odd_Handicap_Casa_Media', 'AvgAHA': 'Odd_Handicap_Visitante_Media',
    'BFEAHH': 'Odd_Handicap_Casa_BetfairEx', 'BFEAHA': 'Odd_Handicap_Visitante_BetfairEx',
    
    # Odds Over/Under 2.5
    'B365>2.5': 'Odd_Mais2.5_Bet365', 'B365<2.5': 'Odd_Menos2.5_Bet365',
    'P>2.5': 'Odd_Mais2.5_Pinnacle', 'P<2.5': 'Odd_Menos2.5_Pinnacle',
    'Max>2.5': 'Odd_Mais2.5_Maxima', 'Max<2.5': 'Odd_Menos2.5_Maxima',
    'Avg>2.5': 'Odd_Mais2.5_Media', 'Avg<2.5': 'Odd_Menos2.5_Media',
    'BFE>2.5': 'Odd_Mais2.5_BetfairEx', 'BFE<2.5': 'Odd_Menos2.5_BetfairEx',
    
    # Odds Fecho (Closing)
    'B365CH': 'Odd_Fecho_Casa_Bet365', 'B365CD': 'Odd_Fecho_Empate_Bet365', 'B365CA': 'Odd_Fecho_Visitante_Bet365',
    'B365C>2.5': 'Odd_Fecho_Mais2.5_Bet365', 'B365C<2.5': 'Odd_Fecho_Menos2.5_Bet365',
    'AHCh': 'Handicap_Asiatico_Linha_Fecho',
    'B365CAHH': 'Odd_Fecho_Handicap_Casa_Bet365', 'B365CAHA': 'Odd_Fecho_Handicap_Visitante_Bet365',
    'PSCH': 'Odd_Fecho_Casa_Pinnacle', 'PSCD': 'Odd_Fecho_Empate_Pinnacle', 'PSCA': 'Odd_Fecho_Visitante_Pinnacle',
    'PC>2.5': 'Odd_Fecho_Mais2.5_Pinnacle', 'PC<2.5': 'Odd_Fecho_Menos2.5_Pinnacle',
    'PCAHH': 'Odd_Fecho_Handicap_Casa_Pinnacle', 'PCAHA': 'Odd_Fecho_Handicap_Visitante_Pinnacle',
    'MaxCH': 'Odd_Fecho_Casa_Maxima', 'MaxCD': 'Odd_Fecho_Empate_Maxima', 'MaxCA': 'Odd_Fecho_Visitante_Maxima',
    'AvgCH': 'Odd_Fecho_Casa_Media', 'AvgCD': 'Odd_Fecho_Empate_Media', 'AvgCA': 'Odd_Fecho_Visitante_Media',
    'MaxC>2.5': 'Odd_Fecho_Mais2.5_Maxima', 'MaxC<2.5': 'Odd_Fecho_Menos2.5_Maxima',
    'AvgC>2.5': 'Odd_Fecho_Mais2.5_Media', 'AvgC<2.5': 'Odd_Fecho_Menos2.5_Media',
    'MaxCAHH': 'Odd_Fecho_Handicap_Casa_Maxima', 'MaxCAHA': 'Odd_Fecho_Handicap_Visitante_Maxima',
    'AvgCAHH': 'Odd_Fecho_Handicap_Casa_Media', 'AvgCAHA': 'Odd_Fecho_Handicap_Visitante_Media',
    
    # Outras Odds Fecho
    'BWCH': 'Odd_Fecho_Casa_BetWin', 'BWCD': 'Odd_Fecho_Empate_BetWin', 'BWCA': 'Odd_Fecho_Visitante_BetWin',
    'IWCH': 'Odd_Fecho_Casa_Interwetten', 'IWCD': 'Odd_Fecho_Empate_Interwetten', 'IWCA': 'Odd_Fecho_Visitante_Interwetten',
    'WHCH': 'Odd_Fecho_Casa_WilliamHill', 'WHCD': 'Odd_Fecho_Empate_WilliamHill', 'WHCA': 'Odd_Fecho_Visitante_WilliamHill',
    'VCCH': 'Odd_Fecho_Casa_VCBet', 'VCCD': 'Odd_Fecho_Empate_VCBet', 'VCCA': 'Odd_Fecho_Visitante_VCBet',
    'BFCH': 'Odd_Fecho_Casa_Betfair', 'BFCD': 'Odd_Fecho_Empate_Betfair', 'BFCA': 'Odd_Fecho_Visitante_Betfair',
    'BFECH': 'Odd_Fecho_Casa_BetfairEx', 'BFECD': 'Odd_Fecho_Empate_BetfairEx', 'BFECA': 'Odd_Fecho_Visitante_BetfairEx',
    'BFEC>2.5': 'Odd_Fecho_Mais2.5_BetfairEx', 'BFEC<2.5': 'Odd_Fecho_Menos2.5_BetfairEx',
    'BFECAHH': 'Odd_Fecho_Handicap_Casa_BetfairEx', 'BFECAHA': 'Odd_Fecho_Handicap_Visitante_BetfairEx',
    'LBCH': 'Odd_Fecho_Casa_Ladbrokes', 'LBCD': 'Odd_Fecho_Empate_Ladbrokes', 'LBCA': 'Odd_Fecho_Visitante_Ladbrokes',
    'BMGMCH': 'Odd_Fecho_Casa_BetMGM', 'BMGMCD': 'Odd_Fecho_Empate_BetMGM', 'BMGMCA': 'Odd_Fecho_Visitante_BetMGM',
    'BVCH': 'Odd_Fecho_Casa_BetVictor', 'BVCD': 'Odd_Fecho_Empate_BetVictor', 'BVCA': 'Odd_Fecho_Visitante_BetVictor',
    'CLCH': 'Odd_Fecho_Casa_Coral', 'CLCD': 'Odd_Fecho_Empate_Coral', 'CLCA': 'Odd_Fecho_Visitante_Coral',
    '1XBCH': 'Odd_Fecho_Casa_1xBet', '1XBCD': 'Odd_Fecho_Empate_1xBet', '1XBCA': 'Odd_Fecho_Visitante_1xBet'
}

df_final = df_final.rename(columns=col_map_pt)

# Guardar
df_final.to_csv("dataset_final_merged.csv", index=False)
print("Ficheiro dataset_final_merged.csv criado com nomes em Português.")

Ficheiro dataset_final_merged.csv criado com nomes em Português.
